In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("HELK JOIN") \
    .master("spark://helk-spark-master:7077") \
    .enableHiveSupport() \
    .getOrCreate()

In [3]:
es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.read.field.as.array.include", "tags")
    .option("es.nodes","helk-elasticsearch:9200")
    .option("es.net.http.auth.user","elastic")
)

In [4]:
security_df = es_reader.load("logs-endpoint-winevent-security-*/")

In [5]:
security_df.createOrReplaceTempView("security_events")

In [6]:
sysmon_df = es_reader.load("logs-endpoint-winevent-sysmon-*/")

In [7]:
sysmon_df.createOrReplaceTempView("sysmon_events")

In [29]:
process_hollowing = spark.sql(
'''
SELECT 
    c.host_name, 
    c.user_name,
    c.process_parent_name,
    a.process_name,
    c.process_name,
    a.logon_type,
    c.user_logon_id AS Sysmon_EID_1_LogonId,
    a.user_logon_id AS Security_EID_4624_TargetLogonId
FROM security_events a
JOIN security_events b
    ON a.user_logon_id = b.user_logon_id
    AND b.event_id = 4688
JOIN sysmon_events c
    ON c.process_name = b.process_name
    AND c.event_id = 1
    AND c.user_name = b.user_name
    AND c.user_logon_id != b.user_logon_id
JOIN sysmon_events d
    ON c.process_guid = d.process_target_guid
    AND d.event_id = 10
    AND (d.process_granted_access & 40) == 40
    AND c.process_name = d.process_target_name
    AND c.process_parent_guid = d.process_guid
WHERE 
    a.event_id = 4624
    AND a.logon_type = 11
    AND a.src_ip_addr is not null
'''
).show()

+------------------+---------+--------------------+------------+------------+----------+--------------------+-------------------------------+
|         host_name|user_name| process_parent_name|process_name|process_name|logon_type|Sysmon_EID_1_LogonId|Security_EID_4624_TargetLogonId|
+------------------+---------+--------------------+------------+------------+----------+--------------------+-------------------------------+
|win10.marvel.local|     thor|processhollowing.exe| svchost.exe|     cmd.exe|        11|             9340634|                         685481|
+------------------+---------+--------------------+------------+------------+----------+--------------------+-------------------------------+

